In [1]:
import nltk
import csv
import ssl ## workaround for my specific authorization issue
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from rake_nltk import Rake


from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from sklearn import svm

from gensim.models import Word2Vec
import time

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import classification_report

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df =pd.read_csv("out_0426.csv")

In [3]:
df_balanced = pd.concat([df[df["stars"]==5].sample(5000), df[df["stars"]==4].sample(5000), \
                         df[df["stars"]==3].sample(10000), df[df["stars"]==2].sample(5000), \
                         df[df["stars"]==1].sample(5000)], axis=0)

#Larger df
sample_size = 10000
# df_balanced = pd.concat([complete_df[df["stars"]==5].sample(sample_size), complete_df[df["stars"]==4].sample(sample_size), \
#                          complete_df[df["stars"]==3].sample(sample_size*2), complete_df[df["stars"]==2].sample(sample_size), \
#                          complete_df[df["stars"]==1].sample(sample_size)], axis=0)
df_balanced.head(10)

,user_id,stars,useful,text,tokenized_text,tokenized_text_string,compound,neg,neu,pos,star_revised
94304,7MS7J_RaU0H3JIkvcHiTIA,5,0,In town for business and just had to come here...,"['town', 'busi', 'come', 'grill', 'octopu', 'd...",town busi come grill octopu disappoint group 5...,0.8402,0.048,0.762,0.190,1
156625,E_Q8-6iU5w8PN3Ox1WvaUg,5,2,Update: We have been Annual passholders at Uni...,"['updat', 'annual', 'passhold', 'univers', 're...",updat annual passhold univers resort 10 year a...,0.9118,0.000,0.792,0.208,1
128891,QnzTUxpyk5YmONZvX6OmsA,5,1,Nice spot at busy downtown! It's very pleasure...,"['nice', 'spot', 'busi', 'downtown', ""'s"", 'pl...",nice spot busi downtown 's pleasur enjoy local...,0.8807,0.000,0.469,0.531,1
84470,gcgjbcU8gVQqsiW18xpyAw,5,0,If you can find a problem with this little gem...,"['find', 'problem', 'littl', 'gem', 'good', 'l...",find problem littl gem good luck servic great ...,0.7676,0.149,0.567,0.284,1
58643,7bJiCvCcXSCYaS0xBWL9qw,5,0,Love octane. Very hipster. Service is a little...,"['love', 'octan', 'hipster', 'servic', 'littl'...",love octan hipster servic littl odd locat litt...,0.7964,0.089,0.584,0.327,1
80793,28G9VujxKPUV-oWxCfI7Og,5,1,We came here on a whim before the Trail of Lig...,"['came', 'whim', 'trail', 'light', 'run', 'gra...",came whim trail light run grab quick pump drin...,0.9559,0.000,0.751,0.249,1
140145,oMQPuvvToJCcXZ3GjOfBMg,5,0,I love this place love the customer service ju...,"['love', 'place', 'love', 'custom', 'servic', ...",love place love custom servic wish regist open...,0.9200,0.051,0.581,0.368,1
31387,3s_Ea-VSgjMtZ3lB-RdJyw,5,0,It can be hard to find good coffee at 6am. So ...,"['hard', 'find', 'good', 'coffe', '6am', 'happ...",hard find good coffe 6am happi found place rea...,0.8020,0.074,0.474,0.453,1
113841,AyfqGVHcXA4CykhAI7CqbQ,5,0,We have been going here since it was at it's o...,"['go', 'sinc', ""'s"", 'origin', 'locat', '25+',...",go sinc 's origin locat 25+ year food alway pe...,0.9432,0.000,0.802,0.198,1
181451,b7TgeGanJ0QQQ8_N8h-khA,5,2,I contacted Mr. Yee to remove some heavy items...,"['contact', 'mr.', 'yee', 'remov', 'heavi', 'i...",contact mr. yee remov heavi item offic donat t...,0.6526,0.000,0.771,0.229,1


In [4]:
len(df_balanced)

30000

In [12]:
df = df_balanced.reset_index()
df.head()

,index,user_id,stars,useful,text,tokenized_text,tokenized_text_string,compound,neg,neu,pos,star_revised,tfidf_vectors
0,94304,7MS7J_RaU0H3JIkvcHiTIA,5,0,In town for business and just had to come here...,"['town', 'busi', 'come', 'grill', 'octopu', 'd...",town busi come grill octopu disappoint group 5...,0.8402,0.048,0.762,0.190,1,"(0, 232)\t2.203417403034236\n (0, 145)\t3.5..."
1,156625,E_Q8-6iU5w8PN3Ox1WvaUg,5,2,Update: We have been Annual passholders at Uni...,"['updat', 'annual', 'passhold', 'univers', 're...",updat annual passhold univers resort 10 year a...,0.9118,0.000,0.792,0.208,1,"(0, 232)\t2.203417403034236\n (0, 145)\t3.5..."
2,128891,QnzTUxpyk5YmONZvX6OmsA,5,1,Nice spot at busy downtown! It's very pleasure...,"['nice', 'spot', 'busi', 'downtown', ""'s"", 'pl...",nice spot busi downtown 's pleasur enjoy local...,0.8807,0.000,0.469,0.531,1,"(0, 232)\t2.203417403034236\n (0, 145)\t3.5..."
3,84470,gcgjbcU8gVQqsiW18xpyAw,5,0,If you can find a problem with this little gem...,"['find', 'problem', 'littl', 'gem', 'good', 'l...",find problem littl gem good luck servic great ...,0.7676,0.149,0.567,0.284,1,"(0, 232)\t2.203417403034236\n (0, 145)\t3.5..."
4,58643,7bJiCvCcXSCYaS0xBWL9qw,5,0,Love octane. Very hipster. Service is a little...,"['love', 'octan', 'hipster', 'servic', 'littl'...",love octan hipster servic littl odd locat litt...,0.7964,0.089,0.584,0.327,1,"(0, 232)\t2.203417403034236\n (0, 145)\t3.5..."


In [13]:
len(df)

30000

## TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
sentences = df["tokenized_text_string"]
# sentences = cleaned_df["text"]

vectorizer = TfidfVectorizer(norm = False, smooth_idf = False, min_df=0.04)
df["tfidf_vectors"] = vectorizer.fit_transform(sentences)
dd = vectorizer.fit_transform(sentences)
ddf =pd.DataFrame(dd.todense())
ddf

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,0.000000,0.0,0.0,0.0,0.000000,2.745258,0.0,0.0,0.000000,0.0,...,3.161375,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,3.717091,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,6.810713,0.0,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,2.324008,3.560061
2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,2.912346,0.00000,0.0,0.0,0.000000,3.689228,0.000000,0.000000
4,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
29996,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
29997,0.000000,0.0,0.0,0.0,0.000000,2.745258,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
29998,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000


## Hashing Vectorizer

In [15]:
from sklearn.feature_extraction.text import HashingVectorizer

sentences = df["tokenized_text_string"]

vectorizer = HashingVectorizer(norm = None, n_features = 100)
df["hashing_vectors"] = vectorizer.fit_transform(sentences)

dd = vectorizer.fit_transform(sentences)
ddh =pd.DataFrame(dd.todense())
ddh

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0
1,-1.0,2.0,1.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0
2,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,-2.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,1.0,1.0
29996,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29997,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,-1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
29998,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,2.0,0.0,1.0,-3.0,0.0,0.0,-1.0,0.0,0.0


## Word2Vec

In [16]:
from gensim.models import Word2Vec
import time
# Skip-gram model (sg = 1)
size = 1000
window = 3
min_count = 3
workers = 3
sg = 1

word2vec_model_file = 'word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(df['tokenized_text']).values
# Train the Word2Vec Model
# w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
w2v_model = Word2Vec(stemmed_tokens, min_count = 3)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

Time taken to train word2vec model: 13.143807888031006


In [17]:
# Load the model from the model file
sg_w2v_model = Word2Vec.load(word2vec_model_file)
# Unique ID of the word
print("Index of the word 'food':")
print(sg_w2v_model.wv.key_to_index["a"])
# Total number of the words 
print(len(sg_w2v_model.wv))
# Print the size of the word2vec vector for one word
print("Length of the vector generated for a word")
print(len(sg_w2v_model.wv.get_vector("a", norm=True)))
# Get the mean for the vectors for an example review
print("Print the length after taking average of all word vectors in a sentence:")
# print(np.mean([sg_w2v_model[token] for token in df['tokenized_text'][0]], axis=0))
print(np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in df['tokenized_text'][11243]], axis=0))

Index of the word 'food':
4
236
Length of the vector generated for a word
100
Print the length after taking average of all word vectors in a sentence:
[-0.05920473 -0.06955818  0.00771281 -0.01127043  0.02047168 -0.01848242
 -0.05373682 -0.087043   -0.03547801 -0.06133188 -0.04361162 -0.06223931
  0.04165291  0.06862416  0.00162734 -0.02178333  0.00491352 -0.02351587
  0.06673656  0.06480017  0.11084878 -0.01309489  0.06298432  0.02089581
 -0.05729011  0.01833747 -0.08483457 -0.04224658  0.03662527  0.07286266
  0.02412803  0.03153522  0.03382394  0.00188893  0.03213536  0.09351709
 -0.09807361  0.05173176 -0.09344398  0.00226236  0.09085862 -0.01248838
  0.10746267  0.01386768 -0.03135406 -0.02895029  0.03438146 -0.05460568
  0.0474746  -0.06599099  0.07150201  0.09477388  0.00456988 -0.01877937
 -0.02880884  0.03043007 -0.07783891 -0.02625781 -0.01338576 -0.1933138
  0.02438299  0.03928252  0.00770022  0.0014645   0.04065358 -0.05068913
  0.02794243  0.05842819 -0.01674876 -0.064794 

In [18]:
# def vecc(stem):
#     return np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in stem], axis=0)
# ddw = df["tokenized_text"].apply(vecc)
def vecc(stem):
    stem_list = []
    for token in stem:
        if token not in sg_w2v_model.wv:
            continue
        else:
            stem_list.append(token)
    return np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in stem_list], axis=0)

In [19]:
ddw = df["tokenized_text"].apply(vecc)
ddw.head()

0    [-0.083830655, -0.079063326, 0.0042689587, -0....
1    [-0.08377923, -0.084036894, 0.0040977756, -0.0...
2    [-0.0851384, -0.08110497, 0.0035085005, -0.017...
3    [-0.08095427, -0.08143296, 0.003490805, -0.019...
4    [-0.082977295, -0.079501145, 0.010650711, -0.0...
Name: tokenized_text, dtype: object

In [20]:
ddw = ddw.tolist()

In [21]:
ddw = pd.DataFrame(ddw)
len(ddw)

30000

## Basic ML DF

In [22]:
ml = df[["stars","compound","neg","neu","pos","star_revised"]]
len(ml)

30000

In [23]:
ml.head()

,stars,compound,neg,neu,pos,star_revised
0,5,0.8402,0.048,0.762,0.190,1
1,5,0.9118,0.000,0.792,0.208,1
2,5,0.8807,0.000,0.469,0.531,1
3,5,0.7676,0.149,0.567,0.284,1
4,5,0.7964,0.089,0.584,0.327,1


## Complete DF

In [24]:
complete_df = pd.concat([ml, ddf, ddh, ddw], axis=1)
len(complete_df)

30000

In [29]:
complete_df.head(10)

,stars,compound,neg,neu,pos,star_revised,0,1,2,3,...,90,91,92,93,94,95,96,97,98,99
0,5,0.8402,0.048,0.762,0.190,1,0.000000,0.0,0.0,0.0,...,0.105875,-0.033451,-0.026138,0.024679,-0.037601,-0.061000,-0.166048,0.052458,-0.118615,0.072526
1,5,0.9118,0.000,0.792,0.208,1,3.717091,0.0,0.0,0.0,...,0.105677,-0.039176,-0.027938,0.023725,-0.036364,-0.061635,-0.172103,0.051059,-0.119155,0.071409
2,5,0.8807,0.000,0.469,0.531,1,0.000000,0.0,0.0,0.0,...,0.117437,-0.028717,-0.022030,0.019610,-0.039178,-0.067642,-0.170202,0.057510,-0.118782,0.075258
3,5,0.7676,0.149,0.567,0.284,1,0.000000,0.0,0.0,0.0,...,0.115316,-0.032853,-0.025235,0.023174,-0.034649,-0.067232,-0.162817,0.054005,-0.118994,0.073094
4,5,0.7964,0.089,0.584,0.327,1,0.000000,0.0,0.0,0.0,...,0.098804,-0.030305,-0.021620,0.028816,-0.040456,-0.065448,-0.178661,0.054761,-0.127731,0.077840
5,5,0.9559,0.000,0.751,0.249,1,0.000000,0.0,0.0,0.0,...,0.112265,-0.037989,-0.027086,0.021109,-0.037258,-0.069845,-0.171487,0.060626,-0.121949,0.077748
6,5,0.9200,0.051,0.581,0.368,1,3.717091,0.0,0.0,0.0,...,0.118221,-0.031973,-0.031838,0.020880,-0.039896,-0.066428,-0.172663,0.054973,-0.114514,0.077466
7,5,0.8020,0.074,0.474,0.453,1,0.000000,0.0,0.0,0.0,...,0.129309,-0.034607,-0.026377,0.011015,-0.026100,-0.058738,-0.162435,0.052339,-0.117939,0.086995
8,5,0.9432,0.000,0.802,0.198,1,0.000000,0.0,0.0,0.0,...,0.116839,-0.033797,-0.018935,0.021851,-0.035891,-0.061580,-0.165350,0.052923,-0.124454,0.078240
9,5,0.6526,0.000,0.771,0.229,1,0.000000,0.0,0.0,0.0,...,0.100198,-0.038118,-0.023491,0.019454,-0.036100,-0.053582,-0.162525,0.058051,-0.120261,0.075983


In [29]:
complete_df = df_balanced
len(complete_df)

30000

In [37]:
# compression_opts = dict(method='zip', archive_name='out.csv')
# complete_df.to_csv('complete.zip', index=False, compression=compression_opts)

In [36]:
complete_df.iloc[:,6:]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.0,0.0,0.0,0.000000,2.745258,0.0,0.0,0.000000,0.0,...,0.105875,-0.033451,-0.026138,0.024679,-0.037601,-0.061000,-0.166048,0.052458,-0.118615,0.072526
1,3.717091,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,6.810713,0.0,...,0.105677,-0.039176,-0.027938,0.023725,-0.036364,-0.061635,-0.172103,0.051059,-0.119155,0.071409
2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.117437,-0.028717,-0.022030,0.019610,-0.039178,-0.067642,-0.170202,0.057510,-0.118782,0.075258
3,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.115316,-0.032853,-0.025235,0.023174,-0.034649,-0.067232,-0.162817,0.054005,-0.118994,0.073094
4,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.098804,-0.030305,-0.021620,0.028816,-0.040456,-0.065448,-0.178661,0.054761,-0.127731,0.077840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.111577,-0.031301,-0.029552,0.023044,-0.041849,-0.070420,-0.166805,0.056540,-0.116551,0.068233
29996,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.082966,-0.024784,-0.015498,0.016486,-0.008023,-0.070252,-0.156827,0.073164,-0.102425,0.054823
29997,0.000000,0.0,0.0,0.0,0.000000,2.745258,0.0,0.0,0.000000,0.0,...,0.110631,-0.036877,-0.030233,0.028657,-0.034333,-0.075313,-0.178338,0.054175,-0.123067,0.073069
29998,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.109490,-0.034383,-0.028117,0.018882,-0.033832,-0.072291,-0.167768,0.059603,-0.119954,0.077736


## 优先测试SVM

## Polarity_Score

In [91]:
X = complete_df[["compound","neg","neu","pos"]]
# X = X.head(10000)
y = complete_df["star_revised"]
# y = y.head(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [42]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = clf.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.5218518518518519
test accuracy: 0.519074074074074


In [43]:
y_test_pred = clf.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.529

In [44]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1015  511  282]
 [ 586  606  569]
 [ 359  549  923]]
Score: 47.11
Classification Report:               precision    recall  f1-score   support

          -1       0.52      0.56      0.54      1808
           0       0.36      0.34      0.35      1761
           1       0.52      0.50      0.51      1831

    accuracy                           0.47      5400
   macro avg       0.47      0.47      0.47      5400
weighted avg       0.47      0.47      0.47      5400



## 只有TF-IDF

In [92]:
X = complete_df.iloc[:,6:262]
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [46]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1221  410  177]
 [ 396 1006  359]
 [ 211  417 1203]]
Score: 63.52
Classification Report:               precision    recall  f1-score   support

          -1       0.67      0.68      0.67      1808
           0       0.55      0.57      0.56      1761
           1       0.69      0.66      0.67      1831

    accuracy                           0.64      5400
   macro avg       0.64      0.63      0.64      5400
weighted avg       0.64      0.64      0.64      5400



In [38]:
from sklearn import svm
clf2 = svm.SVC()
clf2.fit(X_train_scaled, y_train)

y_pred = clf2.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = clf2.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.8063888888888889
test accuracy: 0.6468518518518519


In [41]:
y_test_pred = clf2.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.6453333333333333

## 只有 Hashing

In [42]:
X = complete_df.iloc[:,263:362]
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [47]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1211  436  161]
 [ 382  991  388]
 [ 204  425 1202]]
Score: 63.04
Classification Report:               precision    recall  f1-score   support

          -1       0.67      0.67      0.67      1808
           0       0.54      0.56      0.55      1761
           1       0.69      0.66      0.67      1831

    accuracy                           0.63      5400
   macro avg       0.63      0.63      0.63      5400
weighted avg       0.63      0.63      0.63      5400



In [43]:
from sklearn import svm
clf3 = svm.SVC()
clf3.fit(X_train_scaled, y_train)

y_pred = clf3.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = clf3.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.5549537037037037
test accuracy: 0.5466666666666666


In [44]:
y_test_pred = clf3.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.532

## 只有Word2Vec

In [93]:
X = complete_df.iloc[:,363:462]
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [64]:
type(X)

pandas.core.frame.DataFrame

In [52]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[909 454 445]
 [547 650 564]
 [417 467 947]]
Score: 46.41
Classification Report:               precision    recall  f1-score   support

          -1       0.49      0.50      0.49      1808
           0       0.41      0.37      0.39      1761
           1       0.48      0.52      0.50      1831

    accuracy                           0.46      5400
   macro avg       0.46      0.46      0.46      5400
weighted avg       0.46      0.46      0.46      5400



In [53]:
# Support Vector Machine
from sklearn.svm import SVC
svm = SVC(random_state=101)
svm.fit(X_train_scaled,y_train)
predsvm = svm.predict(X_test_scaled)
print("Confusion Matrix for Support Vector Machines:")
print(confusion_matrix(y_test,predsvm))
print("Score:",round(accuracy_score(y_test,predsvm)*100,2))
print("Classification Report:",classification_report(y_test,predsvm))

Confusion Matrix for Support Vector Machines:
[[989 434 385]
 [570 658 533]
 [448 475 908]]
Score: 47.31
Classification Report:               precision    recall  f1-score   support

          -1       0.49      0.55      0.52      1808
           0       0.42      0.37      0.40      1761
           1       0.50      0.50      0.50      1831

    accuracy                           0.47      5400
   macro avg       0.47      0.47      0.47      5400
weighted avg       0.47      0.47      0.47      5400



In [94]:
y_pred = svm.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = svm.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.4764814814814815
test accuracy: 0.47314814814814815


In [95]:
y_test_pred = svm.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.4723333333333333

## All 

In [103]:
X = complete_df.drop(["stars","star_revised"], axis = 1)
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [69]:
X

,compound,neg,neu,pos,0,1,2,3,4,5,...,90,91,92,93,94,95,96,97,98,99
0,0.8402,0.048,0.762,0.190,0.000000,0.0,0.0,0.0,0.000000,2.745258,...,0.105875,-0.033451,-0.026138,0.024679,-0.037601,-0.061000,-0.166048,0.052458,-0.118615,0.072526
1,0.9118,0.000,0.792,0.208,3.717091,0.0,0.0,0.0,0.000000,0.000000,...,0.105677,-0.039176,-0.027938,0.023725,-0.036364,-0.061635,-0.172103,0.051059,-0.119155,0.071409
2,0.8807,0.000,0.469,0.531,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.117437,-0.028717,-0.022030,0.019610,-0.039178,-0.067642,-0.170202,0.057510,-0.118782,0.075258
3,0.7676,0.149,0.567,0.284,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.115316,-0.032853,-0.025235,0.023174,-0.034649,-0.067232,-0.162817,0.054005,-0.118994,0.073094
4,0.7964,0.089,0.584,0.327,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.098804,-0.030305,-0.021620,0.028816,-0.040456,-0.065448,-0.178661,0.054761,-0.127731,0.077840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.2263,0.079,0.826,0.094,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.111577,-0.031301,-0.029552,0.023044,-0.041849,-0.070420,-0.166805,0.056540,-0.116551,0.068233
29996,0.0000,0.000,1.000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.082966,-0.024784,-0.015498,0.016486,-0.008023,-0.070252,-0.156827,0.073164,-0.102425,0.054823
29997,-0.6369,0.129,0.871,0.000,0.000000,0.0,0.0,0.0,0.000000,2.745258,...,0.110631,-0.036877,-0.030233,0.028657,-0.034333,-0.075313,-0.178338,0.054175,-0.123067,0.073069
29998,0.2235,0.000,0.946,0.054,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.109490,-0.034383,-0.028117,0.018882,-0.033832,-0.072291,-0.167768,0.059603,-0.119954,0.077736


In [104]:
# Support Vector Machine
from sklearn.svm import SVC
svm = SVC(random_state=101)
svm.fit(X_train_scaled,y_train)
predsvm = svm.predict(X_test_scaled)
print("Confusion Matrix for Support Vector Machines:")
print(confusion_matrix(y_test,predsvm))
print("Score:",round(accuracy_score(y_test,predsvm)*100,2))
print("Classification Report:",classification_report(y_test,predsvm))

Confusion Matrix for Support Vector Machines:
[[1237  416  155]
 [ 356  985  420]
 [ 154  324 1353]]
Score: 66.2
Classification Report:               precision    recall  f1-score   support

          -1       0.71      0.68      0.70      1808
           0       0.57      0.56      0.57      1761
           1       0.70      0.74      0.72      1831

    accuracy                           0.66      5400
   macro avg       0.66      0.66      0.66      5400
weighted avg       0.66      0.66      0.66      5400



In [105]:
y_pred = svm.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = svm.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.6712037037037037
test accuracy: 0.6620370370370371


In [106]:
y_test_pred = svm.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.6616666666666666

## 看SVM结果如何了

In [88]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier(max_depth=10)
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1246  360  202]
 [ 482  792  487]
 [ 239  376 1216]]
Score: 60.26
Classification Report:               precision    recall  f1-score   support

          -1       0.63      0.69      0.66      1808
           0       0.52      0.45      0.48      1761
           1       0.64      0.66      0.65      1831

    accuracy                           0.60      5400
   macro avg       0.60      0.60      0.60      5400
weighted avg       0.60      0.60      0.60      5400



In [100]:
tr = []
tes= []
fin=[]

for i in [2,4,6,8,10,12,14,16,18,20]:
    rmfr = RandomForestClassifier(max_depth=i)
    rmfr.fit(X_train_scaled,y_train)

    y_pred = rmfr.predict(X_train_scaled)
    acc_train = (accuracy_score(y_pred,y_train))

    y_pred = rmfr.predict(X_test_scaled)
    acc_test = (accuracy_score(y_pred,y_test))
    
    tr.append(acc_train)
    tes.append(acc_test)

    print('train accuracy: {}'.format(acc_train))
    print('test accuracy: {}'.format(acc_test))
    
    y_test_final = rmfr.predict(scaler.transform(X_final_test))
    acc_test = (accuracy_score(y_test_final,y_final_test))
    fin.append(acc_test)

train accuracy: 0.4387962962962963
test accuracy: 0.44037037037037036
train accuracy: 0.4637962962962963
test accuracy: 0.4559259259259259
train accuracy: 0.5099537037037037
test accuracy: 0.4664814814814815
train accuracy: 0.6019907407407408
test accuracy: 0.4722222222222222
train accuracy: 0.7526851851851852
test accuracy: 0.4674074074074074
train accuracy: 0.8868981481481482
test accuracy: 0.46703703703703703
train accuracy: 0.9660648148148148
test accuracy: 0.4635185185185185
train accuracy: 0.9910185185185185
test accuracy: 0.4614814814814815
train accuracy: 0.9990740740740741
test accuracy: 0.4653703703703704
train accuracy: 0.9997685185185186
test accuracy: 0.4737037037037037
